# Première analyse du dataset

## Quelques informations sur les données

L’objectif est de développer **un modèle performant** pour prédire la **variable Output** en fonction des **autres variables pertinentes**. La plupart des variables ont un nom décrivant l’information contenue.

 

**Quelques informations :**

* Les variables finissant en _M : variables calculées au mois M
* Les variables finissant en _MY : variables calculées au mois M-Y
* Les variables finissant en _XM : variables calculées sur les mois M-X au mois M 



### Librairies pour l'analyse

In [56]:
import pandas as pd
import numpy as np 
import seaborn as sns
import math

# Chargement des données

In [19]:
df = pd.read_csv('Dataset_test_ILB.csv',encoding="latin1",sep=';',decimal=',')
# Encodage latin1 car le fichier n'est pas encodé en UTF8, selection de "," en séparateur décimal

### Analyse du df et de ses composantes

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3015 entries, 0 to 3014
Data columns (total 27 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   ID                                    3015 non-null   int64  
 1   Nombres_adultes                       3005 non-null   float64
 2   Nombre_personnes_foyer                3006 non-null   float64
 3   Age_2eme_personne                     3013 non-null   float64
 4   Note_credit                           3010 non-null   float64
 5   Encours_credit                        2993 non-null   object 
 6   ID_PP                                 3008 non-null   object 
 7   Encours_credit_habitat                3011 non-null   float64
 8   Montant_flux_crediteur_M1             3011 non-null   float64
 9   Montant_obligations                   3011 non-null   float64
 10  Montant_operation_crediteur_DAV_M1    3002 non-null   float64
 11  Montant_operation

In [24]:
df.head()

,ID,Nombres_adultes,Nombre_personnes_foyer,Age_2eme_personne,Note_credit,Encours_credit,ID_PP,Encours_credit_habitat,Montant_flux_crediteur_M1,Montant_obligations,...,Nombre_retrait_carte_M1,"Nombre_retrait_carte_M1,1",Quantité_obligations,Nombre_connexion_appli_6M,Nombre_de_connexion_appli_semaine_6M,Nombre_connexion_site_6M,Duree_connexion_moyenne_6M,Montant_flux_crediteur_M,Montant_pret_banque,Output
0,2911,2.057430,2.070880,91.042848,0.177448,"4715,49819",732f2de9240c404122f8c97b952c4cb8be8ede7f,2772.907651,9005.374276,5897.737602,...,0.160537,0.094456,262736.76140,46.683372,41.184480,48.057303,28.811906,5757.583993,124370.2918,FAUX
1,1398,2.060715,2.193722,90.715413,-0.062708,"6584,320922",1cc77d640ec2c383bef7099ff331d2ae962fd712,7094.588072,5047.824214,55655.446030,...,-0.172851,-0.170338,157962.64670,6.806368,9.210575,12.027955,-8.664351,2825.730247,130015.4600,FAUX
2,1793,1.930472,1.946847,90.588554,-0.094438,"-7792,256696",34d7809e534d6e7d83e1439cd4032201386fe9dd,-7613.883523,-692.852208,2277.193995,...,-0.037880,-0.086625,-26034.01144,3.442761,4.988904,7.072354,-21.135796,1679.601033,130904.7126,FAUX
3,1193,1.963631,1.916677,89.300012,-0.286729,"6118,791869",1539c6f04d9403928ef4e5b662490bd4787ad71d,5874.214001,1408.588664,8280.325647,...,0.994902,0.001148,336491.25330,-6.298359,-7.873857,-9.758806,-6.232009,4059.963478,123729.3778,FAUX
4,1342,2.009754,2.019892,87.706161,0.284325,"-4266,843489",27524f055875f70d102006ee6dcb360e982c4a69,-4459.202674,-889.296675,59977.521970,...,-0.027045,0.080714,-10631.26532,3.056277,4.414500,5.862660,-28.520162,1719.696619,126770.7544,FAUX


La majorité des colonnes qui contiennt des float ont bien été converties, sauf **Encours_credit**
> **Encours_credit**: Argent qui a été prêté mais qui n'a pas encore été remboursé.

In [68]:
df.Encours_credit[:30]

0       4715,49819
1      6584,320922
2     -7792,256696
3      6118,791869
4     -4266,843489
5      963,2226795
6      -9868,12031
7      9926,266551
8      2901,144119
9       3508,97993
10     7884,669148
11    -2401,822083
12     14702,44086
13     5286,493638
14    PAS DENCOURS
15    -4648,965858
16     9654,794858
17    -14351,44645
18    -1636,208879
19    -9442,465683
20    -8047,237148
21    -2059,018592
22     3981,916098
23     -2517,34467
24    -6878,210721
25     182,6623636
26    -5287,245058
27    -10360,55858
28      11300,4433
29    -3482,976268
Name: Encours_credit, dtype: object

On comprend mieux pourquoi la conversion n'a pas pu se faire, il faut prendre en compte les personnes qui n'ont pas d'encours credit

#### Formattage d'encours_credit

Pour que l'encours credit garde son sens, je propose de séparer cette colonne en deux colonnes : 

| Encours_credit_bool             | Encours_credit_value |
| ------------------------------- | -------------------- |
| True                            | -1151                |
| False (Remplace 'PAS DENCOURS') | NaN                  |



In [59]:
"""Création de deux array encours_credit_bool & encours_credit_value """
def create_encours_credit_bool_and_value(df):
    encours_credit = df['Encours_credit'].tolist()
    """Fonction qui construit les deux array évoqués ci-dessus"""
    encours_credit_bool = [True]*len(encours_credit)
    encours_credit_value =[np.nan]*len(encours_credit)
    for (i,encours) in enumerate(encours_credit): 
        """Parcourt la liste encours_credit : encours est un str pour le moment"""
        if encours =="PAS DENCOURS":
            encours_credit_bool[i] = False
        else : 
            """Il y a bien un encours, il va falloir convertir le str en float : attention sep ',' """
            try :
                """Il se peut que encours soit un NaN c'est pourquoi on utilise try & except"""
                encours_float = '.'.join(encours.split(',')) # Utilisation de split pour la conversion
                encours_float = float(encours_float)
                encours_credit_value[i] = encours_float
            except : 
                encours_credit_bool[i] = False
                
    """On peut renvoyer les array"""
    return(encours_credit_bool,encours_credit_value)

In [41]:
val = df["Encours_credit"].tolist()[0].split(',')

In [61]:
encours_credit_bool,encours_credit_values = create_encours_credit_bool_and_value(df)

On vérifie si la structure est conforme à ce que nous avions envisagé

In [62]:
encours = pd.DataFrame(zip(encours_credit_bool,encours_credit_values),columns=['EncoursBool','EncoursValues'])

In [67]:
encours.head(15)

,EncoursBool,EncoursValues
0,True,4715.498190
1,True,6584.320922
2,True,-7792.256696
3,True,6118.791869
4,True,-4266.843489
5,True,963.222680
6,True,-9868.120310
7,True,9926.266551
8,True,2901.144119
9,True,3508.979930
